# Poutyne's Tips and Tricks

Poutyne also offers a variety of tools for fine-tuning the information generated during the training, such as colouring the training update message, a progress bar, multi-GPUs, user callbacks interface and a user naming interface for the metrics' names. 

Let's install the latest version of Poutyne and colorama (if they are not already), and import all the needed packages.

In [35]:
%pip install --upgrade poutyne
%pip install --upgrade colorama
%matplotlib inline
import os
import math

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from torchvision import transforms, utils
from torchvision.datasets.mnist import MNIST

from poutyne import set_seeds, Model, ModelCheckpoint, CSVLogger, Callback, Experiment, SKLearnMetrics

## Hyperparameters, Dataset and Network

In this section, we setup the hyperparameters, dataset and network we will use throughout these tips and tricks. 

### Training Constants

Now, let's set our training constants. We first have the CUDA device used for training if one is present. Second, we set the `train_split` to 0.8 (80%) to use 80% of the dataset for training and 20% for testing the trained model. Third, we set the number of classes (i.e. one for each digit). Finally, we set the batch size (i.e. the number of elements to see before updating the model), the learning rate for the optimizer, and the number of epochs (i.e. the number of times we see the full dataset).

In [36]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

train_split_percent = 0.8

num_classes = 10

batch_size = 32
learning_rate = 0.1
num_epochs = 5

In Poutyne, as we will see in the following sections, you can define your own loss functions and optimizers. However, we can also pass magic strings to use PyTorch's standard optimizers and loss functions. Furthermore, for the optimizer, we can also use a dictionary to set other parameters as the learning rate, for instance, if we don't want the default learning rate.

Here, we initialize the dictionary for our optimizer as well as the string for our loss function. We thus use SGD with the specified learning rate and the cross-entropy loss.

In [37]:
optimizer = dict(optim='sgd', lr=learning_rate) # Could be 'sgd' if we didn't need to change the learning rate.
loss_function = 'cross_entropy'

### Loading the dataset

The following code helps load the MNIST dataset and creates the PyTorch DataLoaders that split our datasets into batches. Then, the train DataLoader shuffles the examples of the training dataset to draw the examples without replacement.

In [38]:
full_train_dataset = MNIST('./datasets', train=True, download=True, transform=transforms.ToTensor())
test_dataset = MNIST('./datasets', train=False, download=True, transform=transforms.ToTensor())

num_data = len(full_train_dataset)
train_length = int(math.floor(train_split_percent * num_data))
valid_length = num_data - train_length

train_dataset, valid_dataset = random_split(full_train_dataset, 
                                            [train_length, valid_length],
                                            generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2)

In [39]:
len(train_dataset), len(valid_dataset)

(48000, 12000)

### Initializing the Network

We initialize a simple convolutional neural network.

In [40]:
def create_network():
    return nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(0.25),
        nn.Flatten(),
        nn.Linear(32*7*7, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, num_classes)
    )

## Vanilla Usage

The following code trains our network in the simplest way possible with Poutyne. We use the accuracy metric so that we can see the performance during training.

In [7]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=['accuracy'], 
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 5.92s loss: 0.386890 acc: 87.452083 val_loss: 0.084405 val_acc: 97.491667
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.33s loss: 0.126130 acc: 96.168750 val_loss: 0.063451 val_acc: 97.991667
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.29s loss: 0.095663 acc: 97.112500 val_loss: 0.054529 val_acc: 98.291667
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.23s loss: 0.081406 acc: 97.514583 val_loss: 0.050878 val_acc: 98.441667
Epoch: 5/5 Train steps: 1500 Val steps: 375 5.92s loss: 0.072686 acc: 97.827083 val_loss: 0.045625 val_acc: 98.600000
Test steps: 313 0.76s test_loss: 0.036329 test_acc: 98.760000                                                   


## Initilalizing Your Optimizer and Loss Function Yourself

Instead of using magic strings for the optimizer and the loss function, it's quite easy to initialize your own and pass them to Poutyne.

In [8]:
# Instantiating our network
network = create_network()

# Instantiating our loss function and optimizer
own_optimizer = optim.SGD(network.parameters(), lr=learning_rate)
own_loss_function = nn.CrossEntropyLoss()

# Poutyne Model on GPU
model = Model(network, own_optimizer, own_loss_function, 
              batch_metrics=['accuracy'], 
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.35s loss: 0.377052 acc: 87.650000 val_loss: 0.103967 val_acc: 96.783333
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.40s loss: 0.123300 acc: 96.245833 val_loss: 0.066045 val_acc: 98.075000
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.29s loss: 0.097580 acc: 97.058333 val_loss: 0.052810 val_acc: 98.508333
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.26s loss: 0.080322 acc: 97.583333 val_loss: 0.045271 val_acc: 98.708333
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.20s loss: 0.070436 acc: 97.804167 val_loss: 0.045102 val_acc: 98.641667
Test steps: 313 0.72s test_loss: 0.031262 test_acc: 98.930000                                                   


## Bypassing PyTorch DataLoaders

Above, we defined DataLoaders for our datasets. However, with Poutyne, it is not strictly necessary since it provides the [`fit_dataset`](https://poutyne.org/model.html#poutyne.Model.fit_dataset) and [`evaluate_dataset`](https://poutyne.org/model.html#poutyne.Model.evaluate_dataset) methods to which you can pass the necessary parameters such as the batch size. Under the hood, Poutyne initializes the DataLoaders for you.

In [9]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=['accuracy'], 
              device=device)

# Train
model.fit_dataset(train_dataset, 
                  valid_dataset, 
                  epochs=num_epochs, 
                  batch_size=batch_size, 
                  num_workers=2)

# Test
test_loss, test_acc = model.evaluate_dataset(test_dataset, 
                                             batch_size=batch_size, 
                                             num_workers=2)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.35s loss: 0.367043 acc: 88.095833 val_loss: 0.084185 val_acc: 97.541667
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.18s loss: 0.124308 acc: 96.256250 val_loss: 0.068236 val_acc: 98.033333
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.47s loss: 0.098265 acc: 97.043750 val_loss: 0.060035 val_acc: 98.291667
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.32s loss: 0.081726 acc: 97.487500 val_loss: 0.055153 val_acc: 98.416667
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.03s loss: 0.073448 acc: 97.689583 val_loss: 0.049198 val_acc: 98.558333
Test steps: 313 0.72s test_loss: 0.037922 test_acc: 98.700000                                                  


## Using Callbacks

One nice feature of Poutyne is [callbacks](https://poutyne.org/callbacks.html). Callbacks allow doing actions during the training of the neural network. In the following example, we use three callbacks. The first that saves the latest weights in a file to be able to continue the optimization at the end of training if more epochs are needed. The second that saves the best weights according to the performance on the validation dataset. The last that saves the displayed logs into a TSV file.

In [22]:
# Saves everything into saves/lstm_unidirectional
save_path = "saves/convnet_mnist"
os.makedirs(save_path, exist_ok=True)

callbacks = [
    # Save the latest weights to be able to continue the optimization at the end for more epochs.
    ModelCheckpoint(os.path.join(save_path, 'last_epoch.ckpt')),

    # Save the weights in a new file when the current model is better than all previous models.
    ModelCheckpoint(os.path.join(save_path, 'best_epoch_{epoch}.ckpt'), monitor='val_acc', mode='max', 
                    save_best_only=True, restore_best=True, verbose=True),

    # Save the losses and accuracies for each epoch in a TSV.
    CSVLogger(os.path.join(save_path, 'log.tsv'), separator='\t'),
]

In [23]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function, 
              batch_metrics=['accuracy'],
              device=device)

# Train
model.fit_generator(train_loader,
                    valid_loader,
                    epochs=num_epochs,
                    callbacks=callbacks)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.73s loss: 0.362374 acc: 88.189583 val_loss: 0.085186 val_acc: 97.383333
Epoch 1: val_acc improved from -inf to 97.38333, saving file to saves/convnet_mnist/best_epoch_1.ckpt
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.44s loss: 0.127077 acc: 96.181250 val_loss: 0.060619 val_acc: 98.291667
Epoch 2: val_acc improved from 97.38333 to 98.29167, saving file to saves/convnet_mnist/best_epoch_2.ckpt
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.33s loss: 0.096829 acc: 97.004167 val_loss: 0.055416 val_acc: 98.350000
Epoch 3: val_acc improved from 98.29167 to 98.35000, saving file to saves/convnet_mnist/best_epoch_3.ckpt
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.24s loss: 0.082727 acc: 97.564583 val_loss: 0.047692 val_acc: 98.591667
Epoch 4: val_acc improved from 98.35000 to 98.59167, saving file to saves/convnet_mnist/best_epoch_4.ckpt
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.37s loss: 0.075551 acc: 97.735417 val_loss: 0.042950 val_acc: 

## Making Your Own Callback

While Poutyne provides a great number of [predefined callbacks](https://poutyne.org/callbacks.html), it is sometimes useful to make your own callback.

In the following example, we want to see the effect of temperature on the optimization of our neural network. To do so, we either increase or decrease the temperature during the optimization. As one can see in the result, temperature either as no effect or has a detrimental effect on the performance of the neural network. This is so because the temperature has for effect to artificially changing the learning rates. Since we have found the right learning rate, increasing or decreasing, it shows no improvement on the results.

In [10]:
class CrossEntropyLossWithTemperature(nn.Module):
    """
    This loss module is the cross-entropy loss function
    with temperature. It divides the logits by a temperature
    value before computing the cross-entropy loss.

    Args:
        initial_temperature (float): The initial value of the temperature.
    """

    def __init__(self, initial_temperature):
        super().__init__()
        self.temperature = initial_temperature
        self.celoss = nn.CrossEntropyLoss()

    def forward(self, y_pred, y_true):
        y_pred = y_pred / self.temperature
        return self.celoss(y_pred, y_true)

In [11]:
class TemperatureCallback(Callback):
    """
    This callback multiply the loss temperature with a decay before
    each batch.

    Args:
        celoss_with_temp (CrossEntropyLossWithTemperature): the loss module.
        decay (float): The value of the temperature decay.
    """
    def __init__(self, celoss_with_temp, decay):
        super().__init__()
        self.celoss_with_temp = celoss_with_temp
        self.decay = decay

    def on_train_batch_begin(self, batch, logs):
        self.celoss_with_temp.temperature *= self.decay

So our loss function will be the cross-entropy with temperature with an initial temperature of `0.1` and a temperature decay of `1.0008`.

In [12]:
custom_loss_function = CrossEntropyLossWithTemperature(0.1)
callbacks = [TemperatureCallback(custom_loss_function, 1.0008)]

Now let's test our training loop for one epoch using the accuracy as the batch metric.

In [13]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, custom_loss_function, 
              batch_metrics=['accuracy'],
              device=device)

# Train
model.fit_generator(train_loader,
                    valid_loader,
                    epochs=num_epochs,
                    callbacks=callbacks)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.01s loss: 0.465483 acc: 85.233333 val_loss: 0.081043 val_acc: 97.533333
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.03s loss: 0.133980 acc: 96.070833 val_loss: 0.062399 val_acc: 98.191667
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.21s loss: 0.107722 acc: 96.822917 val_loss: 0.058225 val_acc: 98.316667
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.19s loss: 0.109482 acc: 96.829167 val_loss: 0.063321 val_acc: 98.200000
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.27s loss: 0.125524 acc: 96.508333 val_loss: 0.075388 val_acc: 98.008333
Test steps: 313 0.73s test_loss: 0.066517 test_acc: 98.250000                                                   


## Using Experiment

Most of the time, when using Poutyne (or even Pytorch in general), we will find ourselves in an iterative model hyperparameters finetuning loop. For efficient model search, we will usually wish to save our best performing models, their training and testing statistics and even sometimes wish to retrain an already trained model for further tuning. All of the above can be easily implemented with the flexibility of Poutyne Callbacks, but having to define and initialize each and every Callback object we wish for our model quickly feels cumbersome.

This is why Poutyne provides an [Experiment class](https://poutyne.org/experiment.html), which aims specifically at enabling quick model iteration search, while not sacrificing the quality of a single experiment - statistics logging, best models saving, etc. Experiment is actually a simple wrapper between a PyTorch network and Poutyne's core Callback objects for logging and saving. Given a working directory where to output the various logging files and a PyTorch network, the Experiment class reduces the whole training loop to a single line.

The following code uses [Poutyne's Experiment class](https://poutyne.org/experiment.html) to train a network for 5 epochs. The code is quite simpler than the code in the Poutyne Callbacks section while doing more (only a few lines). Once trained for 5 epochs, it is then possible to resume the optimization at the 5th epoch for 5 more epochs until the 10th epoch using the same function.

In [47]:
def experiment_train(network, name, epochs=5):
    """
    This function creates a Poutyne Experiment, trains the input module
    on the train loader and then tests its performance on the test loader.
    All training and testing statistics are saved, as well as best model
    checkpoints.
    
    Args:
        network (torch.nn.Module): The neural network to train.
        working_directory (str): The directory where to output files to save.
        epochs (int): The number of epochs. (Default: 5)
    """
    # Everything is going to be saved in ./saves/{name}.
    save_path = os.path.join('saves', name)

    # Poutyne Experiment
    expt = Experiment(save_path, network, optimizer=optimizer, task='classif', device=device)

    # Train
    expt.train(train_loader, valid_loader, epochs=epochs)

    # Test
    expt.test(test_loader)

In [48]:
network = create_network()
experiment_train(network, 'convnet_mnist_experiment', epochs=5)

Epoch: 1/5 Train steps: 1500 Val steps: 375 8.24s loss: 0.391692 acc: 87.477083 fscore_micro: 0.874771 val_loss: 0.094987 val_acc: 97.133333 val_fscore_micro: 0.971333
Epoch 1: val_acc improved from -inf to 97.13333, saving file to saves/convnet_mnist_experiment/checkpoint_epoch_1.ckpt
Epoch: 2/5 Train steps: 1500 Val steps: 375 8.12s loss: 0.135298 acc: 95.875000 fscore_micro: 0.958750 val_loss: 0.068016 val_acc: 98.016667 val_fscore_micro: 0.980167
Epoch 2: val_acc improved from 97.13333 to 98.01667, saving file to saves/convnet_mnist_experiment/checkpoint_epoch_2.ckpt
Epoch: 3/5 Train steps: 1500 Val steps: 375 8.17s loss: 0.103265 acc: 96.818750 fscore_micro: 0.968188 val_loss: 0.063111 val_acc: 98.116667 val_fscore_micro: 0.981167
Epoch 3: val_acc improved from 98.01667 to 98.11667, saving file to saves/convnet_mnist_experiment/checkpoint_epoch_3.ckpt
Epoch: 4/5 Train steps: 1500 Val steps: 375 7.99s loss: 0.090514 acc: 97.268750 fscore_micro: 0.972687 val_loss: 0.049040 val_acc: 

Notice how setting `task='classif'` when instantiating `Experiment` adds for use our loss function, the batch metric accuracy, the epoch metric F1 and set up callbacks that use them. If you wish, you still can use your own loss function and metrics instead of passing this argument.

We have trained for 5 epochs, let's now resume training for another 5 epochs for a total of 10 epochs. Notice that we reinstantiate the network. Experiment will load back the weights for us and resume training.

In [49]:
network = create_network()
experiment_train(network, 'convnet_mnist_experiment', epochs=10)

Loading weights from saves/convnet_mnist_experiment/checkpoint.ckpt and starting at epoch 6.
Loading optimizer state from saves/convnet_mnist_experiment/checkpoint.optim and starting at epoch 6.
Epoch:  6/10 Train steps: 1500 Val steps: 375 8.35s loss: 0.060369 acc: 98.133333 fscore_micro: 0.981333 val_loss: 0.044301 val_acc: 98.750000 val_fscore_micro: 0.987500
Epoch 6: val_acc improved from 98.47500 to 98.75000, saving file to saves/convnet_mnist_experiment/checkpoint_epoch_6.ckpt
Epoch:  7/10 Train steps: 1500 Val steps: 375 8.25s loss: 0.049694 acc: 98.452083 fscore_micro: 0.984521 val_loss: 0.046923 val_acc: 98.691667 val_fscore_micro: 0.986917
Epoch:  8/10 Train steps: 1500 Val steps: 375 7.98s loss: 0.045632 acc: 98.622917 fscore_micro: 0.986229 val_loss: 0.052571 val_acc: 98.558333 val_fscore_micro: 0.985583
Epoch:  9/10 Train steps: 1500 Val steps: 375 7.97s loss: 0.042206 acc: 98.710417 fscore_micro: 0.987104 val_loss: 0.042196 val_acc: 98.808333 val_fscore_micro: 0.988083
Ep

## Coloring

Also, Poutyne use by default a coloring template of the training step when the package `colorama` is installed.
One could either remove the coloring (`progress_options=dict(coloring=False)`) or set a different coloring template using the fields:
`text_color`, `ratio_color`, `metric_value_color`, `time_color` and `progress_bar_color`.
If a field is not specified, the default color will be used.
[See available colors in colorama's source code](https://github.com/tartley/colorama/blob/9946cfb/colorama/ansi.py#L49).

Here an example where we set the `text_color` to RED and the `progress_bar_color` to LIGHTGREEN_EX.

In [14]:
progress_options = dict(
    coloring=dict(text_color="RED", progress_bar_color="LIGHTGREEN_EX")
)

In [15]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function, 
              batch_metrics=['accuracy'],
              device=device)

# Train
model.fit_generator(train_loader, 
                    valid_loader, 
                    epochs=num_epochs, 
                    progress_options=progress_options)

# Test
test_loss, test_acc = model.evaluate_generator(test_loader, 
                                               progress_options=progress_options)

Epoch: 1/5 Train steps: 1500 Val steps: 375 6.27s loss: 0.353306 acc: 88.485417 val_loss: 0.092008 val_acc: 97.275000
Epoch: 2/5 Train steps: 1500 Val steps: 375 6.17s loss: 0.131179 acc: 95.954167 val_loss: 0.074757 val_acc: 97.750000
Epoch: 3/5 Train steps: 1500 Val steps: 375 6.31s loss: 0.103332 acc: 96.918750 val_loss: 0.054675 val_acc: 98.300000
Epoch: 4/5 Train steps: 1500 Val steps: 375 6.08s loss: 0.086977 acc: 97.312500 val_loss: 0.049309 val_acc: 98.516667
Epoch: 5/5 Train steps: 1500 Val steps: 375 6.18s loss: 0.077654 acc: 97.633333 val_loss: 0.041916 val_acc: 98.725000
Test steps: 313 0.77s test_loss: 0.030876 test_acc: 99.000000                                                   


## Epoch Metrics
It's also possible to used epoch metrics such as [`F1-score`](https://poutyne.org/metrics.html#poutyne.FBeta). You could also define your own epoch metric using the [`EpochMetric`](https://poutyne.org/metrics.html#epoch-metric-interface) interface.

In [16]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=['accuracy'], 
              epoch_metrics=['f1'], 
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, (test_acc, test_f1) = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 8.16s loss: 0.403972 acc: 86.772917 fscore_micro: 0.867729 val_loss: 0.081378 val_acc: 97.675000 val_fscore_micro: 0.976750
Epoch: 2/5 Train steps: 1500 Val steps: 375 8.18s loss: 0.124362 acc: 96.193750 fscore_micro: 0.961937 val_loss: 0.057505 val_acc: 98.375000 val_fscore_micro: 0.983750
Epoch: 3/5 Train steps: 1500 Val steps: 375 8.21s loss: 0.095135 acc: 97.104167 fscore_micro: 0.971042 val_loss: 0.049130 val_acc: 98.625000 val_fscore_micro: 0.986250
Epoch: 4/5 Train steps: 1500 Val steps: 375 7.97s loss: 0.082603 acc: 97.504167 fscore_micro: 0.975042 val_loss: 0.044362 val_acc: 98.766667 val_fscore_micro: 0.987667
Epoch: 5/5 Train steps: 1500 Val steps: 375 7.82s loss: 0.075309 acc: 97.743750 fscore_micro: 0.977437 val_loss: 0.042211 val_acc: 98.733333 val_fscore_micro: 0.987333
Test steps: 313 0.94s test_loss: 0.030675 test_acc: 99.020000 test_fscore_micro: 0.990200             


Furthermore, you could also use the [`SKLearnMetrics`](https://poutyne.org/metrics.html#poutyne.SKLearnMetrics) wrapper to wrap a Scikit-learn metric as an epoch metric. Below, we show how to compute the AUC ROC using the [`SKLearnMetrics`](https://poutyne.org/metrics.html#poutyne.SKLearnMetrics) class.

In [ ]:
def softmax(x, axis=1):
    """
    Compute softmax function.
    """
    e_x = np.exp(x - x.max(axis=axis, keepdims=True))
    return e_x / e_x.sum(axis=axis, keepdims=True)

def roc_auc(y_true, y_pred, **kwargs):
    """
    Since the `roc_auc_score` from Scikit-learn requires normalized probabilities,
    we use the softmax function on the predictions.
    """
    y_pred = softmax(y_pred)
    return roc_auc_score(y_true, y_pred, **kwargs)

# kwargs are keyword arguments we wish to pass to roc_auc.
roc_epoch_metric = SKLearnMetrics(roc_auc, 
                                  kwargs=dict(multi_class='ovr', average='macro'))

In [19]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=['accuracy'], 
              epoch_metrics=['f1', roc_epoch_metric], 
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, (test_acc, test_f1, test_roc) = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 8.29s loss: 0.378403 acc: 87.687500 fscore_micro: 0.876875 roc_auc: 0.991043 val_loss: 0.080758 val_acc: 97.466667 val_fscore_micro: 0.974667 val_roc_auc: 0.999433
Epoch: 2/5 Train steps: 1500 Val steps: 375 8.51s loss: 0.124227 acc: 96.262500 fscore_micro: 0.962625 roc_auc: 0.998729 val_loss: 0.057270 val_acc: 98.275000 val_fscore_micro: 0.982750 val_roc_auc: 0.999701
Epoch: 3/5 Train steps: 1500 Val steps: 375 8.40s loss: 0.096673 acc: 97.066667 fscore_micro: 0.970667 roc_auc: 0.999156 val_loss: 0.062194 val_acc: 98.108333 val_fscore_micro: 0.981083 val_roc_auc: 0.999711
Epoch: 4/5 Train steps: 1500 Val steps: 375 8.45s loss: 0.082043 acc: 97.500000 fscore_micro: 0.975000 roc_auc: 0.999416 val_loss: 0.051070 val_acc: 98.541667 val_fscore_micro: 0.985417 val_roc_auc: 0.999809
Epoch: 5/5 Train steps: 1500 Val steps: 375 8.48s loss: 0.070684 acc: 97.733333 fscore_micro: 0.977333 roc_auc: 0.999577 val_loss: 0.046408 val_acc: 98.625000 val_fscor

## Custom Metric Names

It's also possible to name the metric using a tuple format `(<metric name>, metric)`. That way, it's possible to use multiple times the same metric type (i.e. having micro and macro F1-score).

In [20]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function,
              batch_metrics=[("My accuracy name", 'accuracy')],
              epoch_metrics=[("My f1 name", 'f1')],
              device=device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

# Test
test_loss, (test_acc, test_f1) = model.evaluate_generator(test_loader)

Epoch: 1/5 Train steps: 1500 Val steps: 375 8.21s loss: 0.361928 My accuracy name: 88.272917 My f1 name: 0.882729 val_loss: 0.090292 val_My accuracy name: 97.208333 val_My f1 name: 0.972083
Epoch: 2/5 Train steps: 1500 Val steps: 375 8.35s loss: 0.130258 My accuracy name: 96.054167 My f1 name: 0.960542 val_loss: 0.065651 val_My accuracy name: 98.116667 val_My f1 name: 0.981167
Epoch: 3/5 Train steps: 1500 Val steps: 375 8.28s loss: 0.096897 My accuracy name: 97.022917 My f1 name: 0.970229 val_loss: 0.050718 val_My accuracy name: 98.491667 val_My f1 name: 0.984917
Epoch: 4/5 Train steps: 1500 Val steps: 375 8.04s loss: 0.079877 My accuracy name: 97.612500 My f1 name: 0.976125 val_loss: 0.047217 val_My accuracy name: 98.558333 val_My f1 name: 0.985583
Epoch: 5/5 Train steps: 1500 Val steps: 375 8.25s loss: 0.069439 My accuracy name: 97.843750 My f1 name: 0.978437 val_loss: 0.046242 val_My accuracy name: 98.600000 val_My f1 name: 0.986000
Test steps: 313 0.91s test_loss: 0.035947 test_My 

## Multi-GPUs

Finally, it's also possible to use multi-GPUs for your training either by specifying a list of devices or using the arg `"all"` to take them all.

> Obviously, you need more than one GPUs for that option.

In our case here, multi-gpus takes more time because the task is not big enough to profit from multi-gpus.

### With a Single GPU

In [21]:
# Instantiating our network
network = create_network()

# Poutyne Model on GPU
model = Model(network, optimizer, loss_function, 
              batch_metrics=['accuracy'],
              device="all")

# Train
model.fit_generator(train_loader, valid_loader, epochs=num_epochs)

Epoch: 1/5 Train steps: 1500 Val steps: 375 13.00s loss: 0.366883 acc: 88.293750 val_loss: 0.101252 val_acc: 96.675000
Epoch: 2/5 Train steps: 1500 Val steps: 375 11.26s loss: 0.135737 acc: 95.912500 val_loss: 0.074831 val_acc: 97.833333
Epoch: 3/5 Train steps: 1500 Val steps: 375 11.22s loss: 0.103911 acc: 96.941667 val_loss: 0.058809 val_acc: 98.225000
Epoch: 4/5 Train steps: 1500 Val steps: 375 11.44s loss: 0.090600 acc: 97.306250 val_loss: 0.058117 val_acc: 98.316667
Epoch: 5/5 Train steps: 1500 Val steps: 375 11.22s loss: 0.080267 acc: 97.552083 val_loss: 0.048293 val_acc: 98.608333


[{'epoch': 1,
  'loss': 0.3668832160917421,
  'time': 13.000929268077016,
  'acc': 88.29375,
  'val_loss': 0.10125183857356508,
  'val_acc': 96.675},
 {'epoch': 2,
  'loss': 0.13573715605431547,
  'time': 11.255656457971781,
  'acc': 95.9125,
  'val_loss': 0.0748305463741223,
  'val_acc': 97.83333333333333},
 {'epoch': 3,
  'loss': 0.10391089688939974,
  'time': 11.217931404709816,
  'acc': 96.94166666666666,
  'val_loss': 0.05880866790888831,
  'val_acc': 98.225},
 {'epoch': 4,
  'loss': 0.09060041940018224,
  'time': 11.444195121061057,
  'acc': 97.30625,
  'val_loss': 0.058117124153844395,
  'val_acc': 98.31666666666666},
 {'epoch': 5,
  'loss': 0.08026705734905167,
  'time': 11.219108303077519,
  'acc': 97.55208333333333,
  'val_loss': 0.04829291540489066,
  'val_acc': 98.60833333333333}]